In [1]:
!pip install -r requirements.txt

Looking in indexes: https://mirrors.aliyun.com/pypi/simple
Ignoring openjtalk: markers 'sys_platform != "linux"' don't match your environment
DEPRECATION: pytorch-lightning 1.7.7 has a non-standard dependency specifier torch>=1.9.*. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [2]:
import os
import shutil
import urllib.request
from zipfile import ZipFile
from git.repo import Repo
from IPython.display import Audio

In [3]:
def mkdir(path):
    folder = os.path.exists(path)
    if not folder:
        os.makedirs(path)
        print('create a new folder at path ' + path)
    else:
        print('folder at path ' + path + ' has been created')

In [4]:
def create_file(path):
    file = os.path.exists(path)
    if not file:
          with open (path, 'w'):
            print('create a new file at path ' + path)
    else:
          print('file at path ' + path + ' has been created')

In [5]:
def download_and_extract_zip(zip_url, target_folder):
    if any(os.listdir(target_folder)):
        print(target_folder + ' is not a empty directory')
        return 

    zip_filename = "downloaded_file.zip"
    urllib.request.urlretrieve(zip_url, zip_filename)

    with ZipFile(zip_filename, 'r') as zip_ref:
        zip_ref.extractall(target_folder)

    os.remove(zip_filename)
    print('audios zip has been downloaded and extracted to ' + target_folder)

In [6]:
# 创建config.yml
dataset_path = "./Data"
config_yml_path = "./Data/config.yml"
try:
    mkdir(dataset_path)
    from config import Config
    config = Config(config_yml_path)
except SystemExit as msg:
    print(msg)

create a new folder at path ./Data
已根据默认配置文件default_config.yml生成配置文件./Data/config.yml。请按该配置文件的说明进行配置后重新运行。
如无特殊需求，请勿修改default_config.yml或备份该文件。
0


In [7]:
# 初始化项目
from config import config
mkdir(config.resample_config.in_dir)
mkdir(config.resample_config.out_dir)
mkdir(config.dataset_path + "filelists")
create_file(config.preprocess_text_config.transcription_path)
create_file(config.preprocess_text_config.train_path)
create_file(config.preprocess_text_config.val_path)
shutil.copy('./configs/config.json', config.dataset_path)
# TODO: 原始语音文件导入到 './Data/audios/raw'
audios_download_url = "https://pan.ai-hobbyist.org/d/Genshin%20Datasets/%E4%B8%AD%E6%96%87%20-%20Chinese/%E5%88%86%E8%A7%92%E8%89%B2%20-%20Single/%E8%A7%92%E8%89%B2%E8%AF%AD%E9%9F%B3%20-%20Character/%E8%83%A1%E6%A1%83.zip?sign=xTQgG4LoMK0H2OuaxK5OHnnL9gsObsrXNK2n7AEeFco=:0"
download_and_extract_zip(audios_download_url, config.resample_config.in_dir)
# TODO: 语音文本标注写入到 './Data/filelists/transcripition.list'
!python transcribe_genshin.py --speaker="hutao" --language="ZH"
print('init project successfully')

create a new folder at path Data/audios/raw
create a new folder at path Data/audios/resampled
create a new folder at path Data/filelists
create a new file at path Data/filelists/transcription.list
create a new file at path Data/filelists/train.list
create a new file at path Data/filelists/val.list
audios zip has been downloaded and extracted to Data/audios/raw
Data/audios/raw
transcribe lab texts finished
init project successfully


In [19]:
# 下载bert模型
if not os.path.exists('./bert/chinese-roberta-wwm-ext-large/pytorch_model.bin'):
    shutil.rmtree('./bert/chinese-roberta-wwm-ext-large')
    Repo.clone_from('https://huggingface.co/hfl/chinese-roberta-wwm-ext-large.git',
                    to_path='./bert/chinese-roberta-wwm-ext-large',
                    branch='main')
    print('download chinese bert model finished')
else:
    print('chinese bert model has been downloaded')

if not os.path.exists('./bert/deberta-v2-large-japanese-char-wwm/pytorch_model.bin'):
    shutil.rmtree('./bert/deberta-v2-large-japanese-char-wwm')
    Repo.clone_from('https://huggingface.co/ku-nlp/deberta-v2-large-japanese-char-wwm',
                    to_path='./bert/deberta-v2-large-japanese-char-wwm',
                    branch='main')
    print('download japanese bert model finished')
else:
    print('japanese bert model has been downloaded')


if not os.path.exists('./bert/deberta-v3-large/pytorch_model.bin'):
    shutil.rmtree('./bert/deberta-v3-large')
    Repo.clone_from('https://huggingface.co/microsoft/deberta-v3-large.git',
                    to_path='./bert/deberta-v3-large',
                    branch='main')
    print('download english bert model finished')
else:
    print('english bert model has been downloaded')

chinese bert model has been downloaded
japanese bert model has been downloaded
english bert model has been downloaded


In [20]:
# 下载emotional模型
if not os.path.exists('./emotional/wav2vec2-large-robust-12-ft-emotion-msp-dim/pytorch_model.bin'):
    shutil.rmtree('./emotional/wav2vec2-large-robust-12-ft-emotion-msp-dim')
    Repo.clone_from('https://huggingface.co/audeering/wav2vec2-large-robust-12-ft-emotion-msp-dim.git',
                    to_path='./emotional/wav2vec2-large-robust-12-ft-emotion-msp-dim', 
                    branch='main')
    print('download emotional model finished')
else:
    print('emotional model has been downloaded')

emotional model has been downloaded


In [11]:
# 试听导入音频
audio_path = './Data/audios/raw/vo_BZLQ001_4_hutao_01.wav'
Audio(audio_path, rate=44100)

In [12]:
# 音频数据重采样
!python ./resample.py

574it [02:23,  4.00it/s]
音频重采样完毕!


In [13]:
# 试听重采样导出音频
audio_path = './Data/audios/resampled/vo_BZLQ001_4_hutao_01.wav'
Audio(audio_path, rate=44100)

In [16]:
# 转录语音文本转为拼音
!python ./preprocess_text.py

Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize
  0%|                                                   | 0/574 [00:00<?, ?it/s]Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.980 seconds.
Prefix dict has been built successfully.
100%|████████████████████████████████████████| 574/574 [00:04<00:00, 133.67it/s]
总重复音频数：0，总未找到的音频数:0
训练集和验证集生成完成！


In [21]:
# 生成bert文件
!python ./bert_gen.py

  0%|                                                   | 0/574 [00:00<?, ?it/s]Some weights of the model checkpoint at ./bert/chinese-roberta-wwm-ext-large were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at ./bert/chinese-roberta-wwm-ext-large were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls

In [26]:
# 生成emo文件
!python ./emo_gen.py

Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize
  0%|                                                   | 0/574 [00:00<?, ?it/s]2023-12-10 21:29:07.561117: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-10 21:29:07.640216: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-10 21:29:08.819753: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [30]:
# 执行训练
!python ./train_ms.py

2023-12-10 22:01:46.893805: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-10 22:01:48.019605: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
加载config中的配置localhost
加载config中的配置10086
加载config中的配置1
加载config中的配置0
加载config中的配置0
加载环境变量 
MASTER_ADDR: localhost,
MASTER_PORT: 10086,
WORLD_SIZE: 1,
RANK: 0,
LOCAL_RANK: 0
12-10 22:01:52 INFO     | data_utils.py:62 | Init dataset...
100%|██████████████████████████████████████| 570/570 [00:00<00:00, 40119.37it/s]
12-10 22:01:52 INFO     | data_utils.py:77 | skipped: 0, total: 570
12-10 22:01:52 INFO     | data_utils.py:62 | Init dataset...
100%|██████████████████████████████████████████| 4/4 [00:00<00:00, 19807.81it/s]
12-10 22:01:52 INFO     | data_u

In [ ]:
# 启动tensorboard
!tensorboard --logdir=Data/models

2023-12-10 23:07:12.416608: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-10 23:07:13.451907: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-12-10 23:07:14.929813: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-10 23:07:14.976446: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returnin

In [35]:
# 推理
!python terminal_infer.py -m "./Data/models/G_3000.pth" -c "./Data/config.json" --text="你好，世界" --speaker='hutao' 

| torch.distributed.nn.jit.instantiator | INFO | Created a temporary directory at /tmp/tmp7vf7pa_x
| torch.distributed.nn.jit.instantiator | INFO | Writing /tmp/tmp7vf7pa_x/_remote_module_non_scriptable.py
Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize
Data/models/G_3000.pth
| utils | INFO | Loaded checkpoint 'Data/models/G_3000.pth' (iteration 41)
运行参数：Namespace(config='./Data/config.json', debug=False, length_scale=1, model='./Data/models/G_3000.pth', noise=0.6, noise_w=0.8, sdp=0.2, share=False, speaker='hutao', text='你好，世界')
你好，世界 hutao 0.2 0.6 0.8 1 ZH None 0
Building prefix dict from the default dictionary ...
| jieba | DEBUG | Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
| jieba | DEBUG | Loading model from cache /tmp/jieba.cache
Loading model cost 0.905 seconds.
| jieba | DEBUG | Loading model cost 0.905 seconds.
Prefix d

In [36]:
# 试听导入音频
audio_path = './音频-你好，世界.wav'
Audio(audio_path, rate=44100)